In [88]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
from random import randrange
import math

x_trainFull = torch.load('x_trainStationarysensing.pt')
y_trainFull = torch.load('y_trainStationarysensing.pt')

#x_trainFull = torch.load('x_trainMovingsensing.pt')
#y_trainFull = torch.load('y_trainMovingsensing.pt')

y_trainFull = y_trainFull.flatten()
y_trainFull = y_trainFull.long()

#Data for moving mouse:
x_train = x_trainFull[1:1001]
y_train = y_trainFull[1:1001]
x_test = x_trainFull[2001:2801]
y_test = y_trainFull[2001:2801]
print(x_trainFull.shape)

torch.Size([7026, 3, 40, 40])


In [89]:
#Michael McMahon mam1198

def bfs(ship, botrow, botcol, d, mouserow, mousecol):

    fringe = [(botrow, botcol)]
    #parent: dictionary: keys: child. values: parent
    parent = {}
    dist = np.ones((d, d)) * np.inf
    dist[botrow, botcol] = 0
    path = []
    while len(fringe) != 0:
        neighbors = []
        curr = fringe.pop(0)
        currrow = curr[0]
        currcol = curr[1]
        #check up
        if currrow > 0:
            if ship[currrow-1, currcol] != 1:
                if dist[currrow-1, currcol] == np.inf:
                    dist[currrow-1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow-1, currcol)] = (currrow, currcol)
                    fringe.append((currrow-1, currcol))
        #check down
        if currrow < d-1:
            if ship[currrow+1, currcol] != 1:
                if dist[currrow+1, currcol] == np.inf:
                    dist[currrow+1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow+1, currcol)] = (currrow, currcol)
                    fringe.append((currrow+1, currcol))
        #check left
        if currcol > 0:
            if ship[currrow, currcol-1] != 1:
                if dist[currrow, currcol-1] == np.inf:
                    dist[currrow, currcol-1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol-1)] = (currrow, currcol)
                    fringe.append((currrow, currcol-1))
        #check right
        if currcol < d-1:
            if ship[currrow, currcol+1] != 1:
                if dist[currrow, currcol+1] == np.inf:
                    dist[currrow, currcol+1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol+1)] = (currrow, currcol)
                    fringe.append((currrow, currcol+1))
    #if (mouserow, mousecol) in parent:
    current = (mouserow, mousecol)
    path.append(current)
    while(1):
        if current in parent:
            current = parent[current]
            path.append(current)
        else:
            break
    path.reverse()
    return path

#Returns the number of adjacent open cells for a block
def check_adj(i, j, ship, d):
    k = 0
    #check up
    if i > 0:
        if ship[i-1, j] != 1:
            k += 1
    #check down
    if i < d-1:
        if ship[i+1, j] != 1:
            k += 1
    #check left
    if j > 0:
        if ship[i, j-1] != 1:
            k +=1
    #check right
    if j < d-1:
        if ship[i, j+1] != 1:
            k +=1
    return k

#Build the ship
#d: dimensions, num_mice: number of mice
def construct(d, num_mice):
    ship = np.ones((d, d))
    open_row = randrange(d)
    open_col = randrange(d)
    ship[open_row, open_col] = 0
    c = 0
    a=0
    while(c==0):
        blocked1n = []
        blocked1n_orig = []
        #Identify all currently blocked cells that have exactly one neighbor
        for i in range(d):
            for j in range(d):
                open_neighbors = 0
                curr = ship[i, j]
                if curr == 1:
                    #check up
                    if i > 0:
                        if ship[i-1, j] == 0:
                            open_neighbors += 1
                    #check down
                    if i < d-1:
                        if ship[i+1, j] == 0:
                            open_neighbors += 1
                    #check left
                    if j > 0:
                        if ship[i, j-1] == 0:
                            open_neighbors +=1
                    #check right
                    if j < d-1:
                        if ship[i, j+1] == 0:
                            open_neighbors +=1
                    if open_neighbors == 1:
                        blocked1n.append((i, j))
        blocked1n_orig = blocked1n.copy()
        #blocked1n: list of currently blocked cells that have exactly one open neighbor
        #Pick one at random and open the selected cell
        if len(blocked1n) > 0:
            ropen = randrange(len(blocked1n))
            rcell = blocked1n[ropen]
            ship[rcell[0], rcell[1]] = 0
        if (len(blocked1n_orig)) == 0:
            c+=1
    #Identify dead ends: open cells with one open neighbor
    #open1n: list of open cells with exactly one open neighbor
    open1n = []
    for i in range(d):
        for j in range(d):
            open_neighbors = 0
            curr = ship[i, j]
            if curr == 0:
                #check up
                if i > 0:
                    if ship[i-1, j] == 0:
                        open_neighbors += 1
                #check down
                if i < d-1:
                    if ship[i+1, j] == 0:
                        open_neighbors += 1
                #check left
                if j > 0:
                    if ship[i, j-1] == 0:
                        open_neighbors +=1
                #check right
                if j < d-1:
                    if ship[i, j+1] == 0:
                        open_neighbors +=1
                if open_neighbors == 1:
                    open1n.append((i, j))
    #For approximately half of these cells, pick one of their closed neighbors at random and open it
    #Closed cell = 1, open cell = 0
    for i in range(int(len(open1n)/2)):
        next_ind = randrange(len(open1n))
        d_end = open1n[next_ind]
        dir_to_open = randrange(4)
        found = 0
        for i in range(4):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        break
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        break
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        break
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        break
            dir_to_open = randrange(4)
        open1n.remove(d_end)
    if len(open1n) == 1:
        d_end = open1n[0]
        while(found==0):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        found += 1
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        found += 1
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        found += 1
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        found += 1
            dir_to_open = randrange(4)
        open1n.remove(open1n[0])
    botrow = randrange(d)
    botcol = randrange(d)
    #Bot = 2
    while ship[botrow, botcol] != 0:
        botrow = randrange(d)
        botcol = randrange(d)
    ship[botrow, botcol] = 2
    mouserow = randrange(d)
    mousecol = randrange(d)
    #Mouse = 3
    while ship[mouserow, mousecol] != 0:
        mouserow = randrange(d)
        mousecol = randrange(d)
    ship[mouserow, mousecol] = 3
    if num_mice == 2:
        mouserow2 = randrange(d)
        mousecol2 = randrange(d)
        double = 0
        if ship[mouserow2, mousecol2] == 3:
            #Found a mouse, 4 will represent both mice occupying the same cell
            double = 1
        while ship[mouserow2, mousecol2] != 0:
            mouserow2 = randrange(d)
            mousecol2 = randrange(d)
            if ship[mouserow2, mousecol2] == 3:
                #Found a mouse, 4 will represent both mice occupying the same cell
                double = 1
        if double == 0:
            ship[mouserow2, mousecol2] = 3
        else:
            ship[mouserow2, mousecol2] = 4
    return ship, botrow, botcol, mouserow, mousecol

#Initializes the probability grid for a given ship
def probbuild(ship):
    pcount = 0
    pindices = []
    probgrid = np.zeros((len(ship[0]), len(ship[0])))
    for i in range(len(ship[0])):
        for j in range(len(ship[0])):
            if ship[i, j] == 0 or ship[i, j] == 3:
                pcount += 1
                pindices.append((i, j))
    for i in range(pcount):
        probgrid[pindices[i][0], pindices[i][1]] = 1 / pcount
    return probgrid

#Updates the probabilities in the grid in the case of a beep from P(A) to P(A|B)
#Beep: 1 if there was a beep, 0 if not
#P(A|B) =  P(A)*e^(−α∗(d−1)) / Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1))
#Let Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1)) = Pb
def response(probgrid, a, botrow, botcol, beep):
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    Pb = 0
    #Sum up the products of the possibilities and e^(−α∗(d−1)) for each possible cell as P(B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pb += (probgrid[i, j] * bprob)
                newgrid[i, j] = probgrid[i, j] * bprob
    #Update the probabilities of each cell to P(A|B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pab = probgrid[i,j] * bprob
                newgrid[i, j] = Pab / Pb
    return newgrid

def response_2_mice(probgrid_m1, botrow, botcol, a, beep):
    #Case: beep received, moved to [moverow, movecol], no mouse
    #Botrow, botcol: intial space where the sensor data was received
    newgrid_m1 = np.zeros((len(probgrid_m1[0]), len(probgrid_m1[0])))
    prob_beep = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                bprob = math.e ** (a*(-1)*(d-1))
                prob_beep += 2 * (probgrid_m1[i, j] * bprob)
                #prob_beep -= (math.e ** (a*(-1)*(d-1))) ** 2 + 2 * (math.e ** (a*(-1)*(d-1)) * (1 - (math.e ** (a*(-1)*(d-1))))) * ((probgrid_m1[i, j]) ** 2)
    #Probability that both bots set off a beep
    pBoth = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            pBoth += probgrid_m1[i, j] * math.e ** (a*(-1)*(outDist-1)) * probgrid_m1[x, y] * math.e ** (a*(-1)*(inDist-1))
    prob_beep_final = prob_beep - pBoth
    #Marginalize over mouse 2
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                newPij = 0
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        #print((outDist, inDist))
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            if beep == 1:
                                p_m1andm2_given_B = (1 - (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1)))) * probgrid_m1[i, j] * probgrid_m1[x, y] / prob_beep_final
                                newPij += p_m1andm2_given_B
                            else:
                                #Set m1 probs to prob m1 and m2 given no beep
                                p_m1andm2_given_NB = (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1))) * probgrid_m1[i, j] * probgrid_m1[x, y] / (1 - prob_beep_final)
                                newPij += p_m1andm2_given_NB

                    newgrid_m1[i, j] = newPij

    return newgrid_m1

#Update the probgrid when a bot moves to a cell and the bot isn't there
#Updates each P(Ai) to P(Ai|~M) = P(Ai ^ ~M) / P(~M) = P(Ai) / (1 - P(M))
def move(probgrid, moverow, movecol):
    #Probability that the mouse was in the cell we moved to
    moveprob = probgrid[moverow, movecol]
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                prevprob = probgrid[i, j]
                newgrid[i, j] = prevprob / (1 - moveprob)
    newgrid[moverow, movecol] = 0
    ##print("sum: ", np.sum(newgrid))
    return newgrid

def mouse_availability(mouserow, mousecol, ship):
    available_mouse_cells = [(mouserow, mousecol)]
    if mouserow > 0:
        if ship[mouserow-1, mousecol] == 0:
            available_mouse_cells.append((mouserow-1, mousecol))
    #check down
    if mouserow < len(ship[0])-1:
        if ship[mouserow+1, mousecol] == 0:
            available_mouse_cells.append((mouserow+1, mousecol))
    #check left
    if mousecol > 0:
        if ship[mouserow, mousecol-1] == 0:
            available_mouse_cells.append((mouserow, mousecol-1))
    #check right
    if mousecol < len(ship[0])-1:
        if ship[mouserow, mousecol+1] == 0:
            available_mouse_cells.append((mouserow, mousecol+1))
    return available_mouse_cells

def mousemove(probgrid, ship):
    newprobs = probgrid.copy()
    d = len(probgrid[0])
    #Build available_moves matrix
    available_moves = np.zeros_like(probgrid)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if ship[i, j] != 1:
                k = check_adj(i, j, ship, len(ship[0]))
                available_moves[i, j] = k + 1
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            #Obtain valid neighbors of ship[i, j]
            if ship[i, j] != 1  and ship[i, j] != 2:
                neighbors = []
                if i > 0:
                    if ship[i-1, j] != 1:
                        neighbors.append((i-1, j))
                #check down
                if i < d-1:
                    if ship[i+1, j] != 1:
                        neighbors.append((i+1, j))
                #check left
                if j > 0:
                    if ship[i, j-1] != 1:
                        neighbors.append((i, j-1))
                #check right
                if j < d-1:
                    if ship[i, j+1] != 1:
                        neighbors.append((i, j+1))
                for n in neighbors:
                    #For each neighbor n of [i, j], add the probability that the mouse was in n and moved to [i, j] P(mouse in n) * P(mouse moves to (i, j))
                    #and subtract the probability that the mouse was in [i, j] and moved to n P(mouse in (i, j)) * P(mouse moves to n)
                    if available_moves[n[0], n[1]] > 0:
                        newprobs[i, j] += probgrid[n[0], n[1]] / available_moves[n[0], n[1]]
                    if available_moves[i, j] > 0:
                        newprobs[i, j] -= probgrid[i, j] / available_moves[i, j]
    return newprobs

x_trainStationary = torch.zeros(1, 3, 40, 40)
y_trainStationary = torch.zeros(1, 1)
x_trainMoving = torch.zeros(1, 3, 40, 40)
y_trainMoving = torch.zeros(1, 1)

def bot3(dim, a, mouse_behavior, num_mice, const_res, batch_size):
    #3 2x5x5 torches

    ship = const_res[0]
    botrow = const_res[1]
    botcol = const_res[2]
    mouserow = const_res[3]
    mousecol = const_res[4]
    mfound = 0
    if num_mice == 2:
        for i in range(len(ship[0])):
            for j in range(len(ship[0])):
                if ship[i, j] == 3 and mfound == 0:
                    mouserow = i
                    mousecol = j
                    mfound += 1
                if ship[i, j] == 3 and mfound == 1:
                    mouserow2 = i
                    mousecol2 = j
    probgrid = probbuild(ship)
    #print(ship)
    #print(probgrid)
    found = 0
    round = 1
    t = 0
    pathbreaks = 0
    micefound = 0
    x = torch.zeros(1, 3, dim, dim)
    y = torch.zeros(1, 1)
    while(found == 0):
        #Sense
        yNew = 4
        npNew = np.zeros((3, 40, 40))
        ship[mouserow, mousecol] = 0
        ship[botrow, botcol] = 0
        npNew[0] = ship
        npNew[1] = probgrid
        botLoc = np.zeros((40, 40))
        botLoc[botrow, botcol] = 1
        npNew[2] = botLoc
        xNew = torch.from_numpy(npNew)
        x[x.shape[0] - 1] = xNew
        x.resize_(x.shape[0] + 1, 3, 40, 40)
        y[y.shape[0] - 1] = yNew
        y.resize_(y.shape[0] + 1, 1)
        if x.shape[0] == batch_size:
            return t, x, y
        ship[mouserow, mousecol] = 3
        ship[botrow, botcol] = 2
        dist = abs(mouserow - botrow) + abs(mousecol - botcol)
        beepprob = math.e ** (a*(-1)*(dist-1))
        roll = random.random()
        roll2 = 1
        beepprob2 = 0
        if num_mice == 2:
            dist2 = abs(mouserow - botrow) + abs(mousecol - botcol)
            beepprob2 = math.e ** (a*(-1)*(dist2-1))
            roll2 = random.random()
        t += 1
        if roll < beepprob:
            if (num_mice == 1) or (num_mice == 2 and micefound == 1):
                probgrid = response(probgrid, a, botrow, botcol, 1)
            else:
                probgrid = response_2_mice(probgrid, botrow, botcol, a, 1)
        else:
            if (num_mice == 1) or (num_mice == 2 and micefound == 1):
                probgrid = response(probgrid, a, botrow, botcol, 0)
            else:
                probgrid = response_2_mice(probgrid, botrow, botcol, a, 0)
        if mouse_behavior == 1:
            if mouserow != -1 and mousecol != -1:
                #Stochastic mouse: allow the mouse to move or stay in its cell with uniform probability
                available_mouse_cells = mouse_availability(mouserow, mousecol, ship)
                #print("available mouse cellse: ", available_mouse_cells)
                mouse_move_cell = random.choice(available_mouse_cells)
                ship[mouserow, mousecol] = 0
                ship[mouse_move_cell[0], mouse_move_cell[1]] = 3
                mouserow = mouse_move_cell[0]
                mousecol = mouse_move_cell[1]
                #Now that the mouse has moved, the bot must reevaluate the probgrid considering that the mouse could have moved
                probgrid = mousemove(probgrid, ship)
            if num_mice == 2:
                if mouserow2 != -1 and mousecol2 != -1:
                    available_mouse_cells = mouse_availability(mouserow2, mousecol2, ship)
                    #print("available mouse cellse: ", available_mouse_cells)
                    mouse_move_cell = random.choice(available_mouse_cells)
                    ship[mouserow2, mousecol2] = 0
                    ship[mouse_move_cell[0], mouse_move_cell[1]] = 3
                    mouserow2 = mouse_move_cell[0]
                    mousecol2 = mouse_move_cell[1]
                    probgrid = mousemove(probgrid, ship)
        maxes = []
        for i in range(len(probgrid[0])):
            for j in range(len(probgrid[0])):
                if probgrid[i, j] == np.max(probgrid):
                    maxes.append((i, j))
        mindist = np.inf
        for m in maxes:
            if abs(m[0] - botrow) + abs(m[1] - botcol) < mindist:
                mindist = abs(m[0] - botrow) + abs(m[1] - botcol)
        for m in maxes:
            if abs(m[0] - botrow) + abs(m[1] - botcol) > mindist:
                maxes.remove(m)
        dest = random.choice(maxes)
        path = bfs(ship, botrow, botcol, dim, dest[0], dest[1])
        round += 1
        for i in range(1, len(path)):
            #Check for mouse
            #yNew = 4
            if path[i][0] == botrow - 1:
                #moving up
                yNew = 0
            if path[i][0] == botrow + 1:
                #moving down
                yNew = 1
            if path[i][1] == botcol - 1:
                #left
                yNew = 2
            if path[i][1] == botcol + 1:
                #right
                yNew = 3
            npNew = np.zeros((3, 40, 40))
            ship[mouserow, mousecol] = 0
            ship[botrow, botcol] = 0
            npNew[0] = ship
            npNew[1] = probgrid
            botLoc = np.zeros((40, 40))
            botLoc[botrow, botcol] = 1
            npNew[2] = botLoc
            xNew = torch.from_numpy(npNew)
            x[x.shape[0] - 1] = xNew
            if x.shape[0] == batch_size:
                return t, x, y
            x.resize_(x.shape[0] + 1, 3, 40, 40)
            y[y.shape[0] - 1] = yNew
            y.resize_(y.shape[0] + 1, 1)
            ship[mouserow, mousecol] = 3
            ship[botrow, botcol] = 2
            if ship[path[i][0], path[i][1]] == 3:
                t += 1
                found += 1
                ship[path[i][0], path[i][1]] = 2
                ship[path[i-1][0], path[i-1][1]] = 0
                botrow = path[i][0]
                botcol = path[i][1]
                break
            else:
                ship[path[i][0], path[i][1]] = 2
                ship[path[i-1][0], path[i-1][1]] = 0
                t += 1
                botrow = path[i][0]
                botcol = path[i][1]
                probgrid = move(probgrid, botrow, botcol)
                #print(ship)
                #print(probgrid)
                flat = np.ndarray.flatten(probgrid)
                options = [x for x in flat if x > 0]
                mean = np.mean(options)
                varnum = 0
                for i in options:
                    varnum += (i - mean) ** 2
                verdenom = len(options) - 1
                variance = varnum / verdenom
                if variance < 0.0001 and pathbreaks < 50:
                    pathbreaks += 1
                    break
    x.resize_(x.shape[0] - 1, 3, 40, 40)
    y.resize_(y.shape[0] - 1, 1)
    return t, x, y

constShip = np.array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
 [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
  0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,],
 [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
  0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,],
 [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
  0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
  1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
  0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
  0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
  0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
  1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
  0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
  0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
  1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,],
 [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
  0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
  0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
  0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
  1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
  1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,],
 [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
  0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,]])



In [90]:
class CNN_MNISTNetwork(nn.Module):
  def __init__(self):
    super(CNN_MNISTNetwork, self).__init__()
    #multiple layers

    #1 conv, 1 linear
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 10, kernel_size = 3 , stride = 1)
    self.conv2 = nn.Conv2d(in_channels = 10, out_channels = 16, kernel_size = 3, stride = 1)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.linear1 = nn.Linear(in_features = 36992, out_features = 5)
    self.linear2 = nn.Linear(in_features = 10000, out_features = 5)


  def forward(self, input_tensor):

    #Working
    x = torch.nn.Tanh()(self.conv1(input_tensor))
    x = torch.nn.Tanh()(self.conv2(x))
    x = torch.nn.Tanh()(self.conv3(x))
    x = torch.flatten(x, 1)
    logits = self.linear1(x)

    for t in range(input_tensor.shape[0]): #will be over input_tensor.shape[0]
      #Each datum:
      br = 0
      bc = 0
      for i in range(input_tensor.shape[2]):
        for j in range(input_tensor.shape[2]):
          if input_tensor[t][2][i, j] == 1:
            br = i
            bc = j
            break
      #We have bot location, now check for adjacent blocked cells (in x_train[t][0])

      blocked_moves = np.array([0, 0, 0, 0])
      #0: up, 1: down, 2: left, 3: right
      if br == 0:
        blocked_moves[0] = 1
      if br == input_tensor.shape[2] - 1:
        blocked_moves[1] = 1
      if bc == 0:
        blocked_moves[2] = 1
      if bc == input_tensor.shape[2] - 1:
        blocked_moves[3] = 1
      if br > 0 and blocked_moves[0] == 0:
        if int(input_tensor[t][0][br - 1, bc]) == 1:
          blocked_moves[0] = 1
      if br < input_tensor.shape[2] - 1 and blocked_moves[1] == 0:
        if int(input_tensor[t][0][br + 1, bc]) == 1:
          blocked_moves[1] = 1
      if bc > 0 and blocked_moves[2] == 0:
        if int(input_tensor[t][0][br, bc - 1]) == 1:
          blocked_moves[2] = 1
      if bc < input_tensor.shape[2] - 1 and blocked_moves[3] == 0:
        if int(input_tensor[t][0][br, bc + 1]) == 1:
          blocked_moves[3] = 1
      for b in range(4):
        if blocked_moves[b] == 1:
          logits[t][b] = -float('inf')

    #probabilities = torch.nn.Softmax(dim=1)(logits)
    probabilities = logits
    return logits, probabilities

In [91]:
model = CNN_MNISTNetwork()

In [92]:
def confusion_matrix(model, x, y):
  identification_counts = np.zeros( shape = (5,5), dtype = np.int32 )
  probabilities = model( x )
  print(probabilities[1].shape)
  predicted_classes = torch.argmax( probabilities[1], dim = 1 )
  n = x.shape[0]

  for i in range(n):
    actual_class = int(y[i])
    predicted_class = predicted_classes[i].item()

    identification_counts[actual_class, predicted_class] += 1

  return identification_counts

print( confusion_matrix( model, x_train, y_train ) )

torch.Size([1000, 5])
[[195   0   0   0   0]
 [180  46   0   0   0]
 [122  74   0   0  70]
 [132  49   0   0  60]
 [ 51  12   0   0   9]]


In [93]:
def get_batch( x, y, batch_size ):
  n = x.shape[0]
  batch_indices = random.sample( [ i for i in range(n) ], k = batch_size )
  x_batch = x[ batch_indices ]
  y_batch = y[ batch_indices ]
  return x_batch, y_batch

In [94]:
model = CNN_MNISTNetwork()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam( model.parameters(), lr=0.01 )
batch_size = 30
#x_full, y_full = get_batch(1000)

for epochs in range(12):
  total_loss = 0
  for batch in range( x_train.shape[0] // batch_size ):
    x_batch, y_batch = get_batch(x_train, y_train, batch_size)
    logits, probabilities = model( x_batch )
    loss = loss_function( logits, y_batch )
    #correct_class_probabilities = torch.gather( probabilities, dim = 1, index = torch.reshape( y_batch, (-1,1) ) )
    #loss = torch.mean( -1 * torch.log( correct_class_probabilities ) )

    loss.backward()
    #optimizer.step()
    total_loss += loss.item()
    optimizer.step()
      #total_loss += loss.item()
    optimizer.zero_grad()
  print("Epoch:", epochs)

  print("Average Loss per Data Point:", total_loss )
  #print(probabilities)
  #print(y_batch)
  #print(correct_class_probabilities)
  print(logits.shape)
  #print(logits.shape[1])
  #print(probabilities.shape)

Epoch: 0
Average Loss per Data Point: 293.2999323606491
torch.Size([30, 5])
Epoch: 1
Average Loss per Data Point: 52.222076058387756
torch.Size([30, 5])
Epoch: 2
Average Loss per Data Point: 40.98846995830536
torch.Size([30, 5])
Epoch: 3
Average Loss per Data Point: 35.96426123380661
torch.Size([30, 5])
Epoch: 4
Average Loss per Data Point: 31.52343362569809
torch.Size([30, 5])
Epoch: 5
Average Loss per Data Point: 28.925707638263702
torch.Size([30, 5])
Epoch: 6
Average Loss per Data Point: 27.318379759788513
torch.Size([30, 5])
Epoch: 7
Average Loss per Data Point: 27.224838733673096
torch.Size([30, 5])
Epoch: 8
Average Loss per Data Point: 24.56913161277771
torch.Size([30, 5])
Epoch: 9
Average Loss per Data Point: 24.447101175785065
torch.Size([30, 5])
Epoch: 10
Average Loss per Data Point: 24.641667753458023
torch.Size([30, 5])
Epoch: 11
Average Loss per Data Point: 23.82040348649025
torch.Size([30, 5])


In [95]:
print( confusion_matrix( model, x_train, y_train ) )

torch.Size([1000, 5])
[[104  49  16  21   5]
 [  9 194  11   4   8]
 [  4  36 188  29   9]
 [ 16  24  59 136   6]
 [  4  21  16   5  26]]


In [97]:
print( confusion_matrix( model, x_test, y_test ) )

torch.Size([800, 5])
[[ 53  90  27  35  14]
 [ 21 134   6  12   9]
 [ 15  38  98  26   3]
 [  6  53  45  44   4]
 [  7  25  13   7  15]]


In [98]:
#performance of model on bot 3: Use constant ship, populate with bot and mouse, import bot 3 and helper
#functions, remove bot control, leave mouse control, call model on the current ship and probgrid
#each time to get the action, have the bot perform the action, loop until mouse is found
constShip = np.array([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
 [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
  0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,],
 [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
  0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
  0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,],
 [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
  1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
  0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
  1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
  0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
  0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
  0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
  0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
  0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
  1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
  1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
  0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,],
 [1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
  0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
  1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,],
 [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
  0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,],
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
  0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,],
 [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
  1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
  0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
  1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,],
 [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
  1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,],
 [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
  0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
  1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,]])
botrow = randrange(40)
botcol = randrange(40)
bot_mouse_ship = constShip.copy()
#Bot = 2
while bot_mouse_ship[botrow, botcol] != 0:
    botrow = randrange(40)
    botcol = randrange(40)
bot_mouse_ship[botrow, botcol] = 2
mouserow = randrange(40)
mousecol = randrange(40)
#Mouse = 3
while bot_mouse_ship[mouserow, mousecol] != 0:
    mouserow = randrange(40)
    mousecol = randrange(40)
bot_mouse_ship[mouserow, mousecol] = 3
const_res = [bot_mouse_ship, botrow, botcol, mouserow, mousecol]


In [99]:
def bfs(ship, botrow, botcol, d, mouserow, mousecol):
    fringe = [(botrow, botcol)]
    #parent: dictionary: keys: child. values: parent
    parent = {}
    dist = np.ones((d, d)) * np.inf
    dist[botrow, botcol] = 0
    path = []
    while len(fringe) != 0:
        neighbors = []
        curr = fringe.pop(0)
        currrow = curr[0]
        currcol = curr[1]
        #check up
        if currrow > 0:
            if ship[currrow-1, currcol] != 1:
                if dist[currrow-1, currcol] == np.inf:
                    dist[currrow-1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow-1, currcol)] = (currrow, currcol)
                    fringe.append((currrow-1, currcol))
        #check down
        if currrow < d-1:
            if ship[currrow+1, currcol] != 1:
                if dist[currrow+1, currcol] == np.inf:
                    dist[currrow+1, currcol] = dist[currrow, currcol] + 1
                    parent[(currrow+1, currcol)] = (currrow, currcol)
                    fringe.append((currrow+1, currcol))
        #check left
        if currcol > 0:
            if ship[currrow, currcol-1] != 1:
                if dist[currrow, currcol-1] == np.inf:
                    dist[currrow, currcol-1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol-1)] = (currrow, currcol)
                    fringe.append((currrow, currcol-1))
        #check right
        if currcol < d-1:
            if ship[currrow, currcol+1] != 1:
                if dist[currrow, currcol+1] == np.inf:
                    dist[currrow, currcol+1] = dist[currrow, currcol] + 1
                    parent[(currrow, currcol+1)] = (currrow, currcol)
                    fringe.append((currrow, currcol+1))
    #if (mouserow, mousecol) in parent:
    current = (mouserow, mousecol)
    path.append(current)
    while(1):
        if current in parent:
            current = parent[current]
            path.append(current)
        else:
            break
    path.reverse()
    return path

#Returns the number of adjacent open cells for a block
def check_adj(i, j, ship, d):
    k = 0
    #check up
    if i > 0:
        if ship[i-1, j] != 1:
            k += 1
    #check down
    if i < d-1:
        if ship[i+1, j] != 1:
            k += 1
    #check left
    if j > 0:
        if ship[i, j-1] != 1:
            k +=1
    #check right
    if j < d-1:
        if ship[i, j+1] != 1:
            k +=1
    return k

#Build the ship
#d: dimensions, num_mice: number of mice
def construct(d, num_mice):
    ship = np.ones((d, d))
    open_row = randrange(d)
    open_col = randrange(d)
    ship[open_row, open_col] = 0
    c = 0
    a=0
    while(c==0):
        blocked1n = []
        blocked1n_orig = []
        #Identify all currently blocked cells that have exactly one neighbor
        for i in range(d):
            for j in range(d):
                open_neighbors = 0
                curr = ship[i, j]
                if curr == 1:
                    #check up
                    if i > 0:
                        if ship[i-1, j] == 0:
                            open_neighbors += 1
                    #check down
                    if i < d-1:
                        if ship[i+1, j] == 0:
                            open_neighbors += 1
                    #check left
                    if j > 0:
                        if ship[i, j-1] == 0:
                            open_neighbors +=1
                    #check right
                    if j < d-1:
                        if ship[i, j+1] == 0:
                            open_neighbors +=1
                    if open_neighbors == 1:
                        blocked1n.append((i, j))
        blocked1n_orig = blocked1n.copy()
        #blocked1n: list of currently blocked cells that have exactly one open neighbor
        #Pick one at random and open the selected cell
        if len(blocked1n) > 0:
            ropen = randrange(len(blocked1n))
            rcell = blocked1n[ropen]
            ship[rcell[0], rcell[1]] = 0
        if (len(blocked1n_orig)) == 0:
            c+=1
    #Identify dead ends: open cells with one open neighbor
    #open1n: list of open cells with exactly one open neighbor
    open1n = []
    for i in range(d):
        for j in range(d):
            open_neighbors = 0
            curr = ship[i, j]
            if curr == 0:
                #check up
                if i > 0:
                    if ship[i-1, j] == 0:
                        open_neighbors += 1
                #check down
                if i < d-1:
                    if ship[i+1, j] == 0:
                        open_neighbors += 1
                #check left
                if j > 0:
                    if ship[i, j-1] == 0:
                        open_neighbors +=1
                #check right
                if j < d-1:
                    if ship[i, j+1] == 0:
                        open_neighbors +=1
                if open_neighbors == 1:
                    open1n.append((i, j))
    #For approximately half of these cells, pick one of their closed neighbors at random and open it
    #Closed cell = 1, open cell = 0
    for i in range(int(len(open1n)/2)):
        next_ind = randrange(len(open1n))
        d_end = open1n[next_ind]
        dir_to_open = randrange(4)
        found = 0
        for i in range(4):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        break
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        break
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        break
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        break
            dir_to_open = randrange(4)
        open1n.remove(d_end)
    if len(open1n) == 1:
        d_end = open1n[0]
        while(found==0):
            #check up
            if dir_to_open == 0:
                if d_end[0] > 0:
                    if ship[d_end[0]-1, d_end[1]] == 1:
                        ship[d_end[0]-1, d_end[1]] = 0
                        found += 1
            #check down
            if dir_to_open == 1:
                if d_end[0] < d-1:
                    if ship[d_end[0]+1, d_end[1]] == 1:
                        ship[d_end[0]+1, d_end[1]] = 0
                        found += 1
            #check left
            if dir_to_open == 2:
                if d_end[1] > 0:
                    if ship[d_end[0], d_end[1]-1] == 1:
                        ship[d_end[0], d_end[1]-1] = 0
                        found += 1
            #check right
            if dir_to_open == 3:
                if d_end[1] < d-1:
                    if ship[d_end[0], d_end[1]+1] == 1:
                        ship[d_end[0], d_end[1]+1] = 0
                        found += 1
            dir_to_open = randrange(4)
        open1n.remove(open1n[0])
    botrow = randrange(d)
    botcol = randrange(d)
    #Bot = 2
    while ship[botrow, botcol] != 0:
        botrow = randrange(d)
        botcol = randrange(d)
    ship[botrow, botcol] = 2
    mouserow = randrange(d)
    mousecol = randrange(d)
    #Mouse = 3
    while ship[mouserow, mousecol] != 0:
        mouserow = randrange(d)
        mousecol = randrange(d)
    ship[mouserow, mousecol] = 3
    if num_mice == 2:
        mouserow2 = randrange(d)
        mousecol2 = randrange(d)
        double = 0
        if ship[mouserow2, mousecol2] == 3:
            #Found a mouse, 4 will represent both mice occupying the same cell
            double = 1
        while ship[mouserow2, mousecol2] != 0:
            mouserow2 = randrange(d)
            mousecol2 = randrange(d)
            if ship[mouserow2, mousecol2] == 3:
                #Found a mouse, 4 will represent both mice occupying the same cell
                double = 1
        if double == 0:
            ship[mouserow2, mousecol2] = 3
        else:
            ship[mouserow2, mousecol2] = 4
    return ship, botrow, botcol, mouserow, mousecol

#Initializes the probability grid for a given ship
def probbuild(ship):
    pcount = 0
    pindices = []
    probgrid = np.zeros((len(ship[0]), len(ship[0])))
    for i in range(len(ship[0])):
        for j in range(len(ship[0])):
            if ship[i, j] == 0 or ship[i, j] == 3:
                pcount += 1
                pindices.append((i, j))
    for i in range(pcount):
        probgrid[pindices[i][0], pindices[i][1]] = 1 / pcount
    return probgrid

#Updates the probabilities in the grid in the case of a beep from P(A) to P(A|B)
#Beep: 1 if there was a beep, 0 if not
#P(A|B) =  P(A)*e^(−α∗(d−1)) / Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1))
#Let Σ(i=1, pcount) P(Ai)*e^(−α∗(d−1)) = Pb
def response(probgrid, a, botrow, botcol, beep):
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    Pb = 0
    #Sum up the products of the possibilities and e^(−α∗(d−1)) for each possible cell as P(B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pb += (probgrid[i, j] * bprob)
                newgrid[i, j] = probgrid[i, j] * bprob
    #Update the probabilities of each cell to P(A|B)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                if beep == 1:
                    bprob = math.e ** (a*(-1)*(d-1))
                else:
                    bprob = 1 - (math.e ** (a*(-1)*(d-1)))
                Pab = probgrid[i,j] * bprob
                newgrid[i, j] = Pab / Pb
    return newgrid

def response_2_mice(probgrid_m1, botrow, botcol, a, beep):
    #Case: beep received, moved to [moverow, movecol], no mouse
    #Botrow, botcol: intial space where the sensor data was received
    newgrid_m1 = np.zeros((len(probgrid_m1[0]), len(probgrid_m1[0])))
    prob_beep = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                d = abs(i - botrow) + abs(j - botcol)
                bprob = math.e ** (a*(-1)*(d-1))
                prob_beep += 2 * (probgrid_m1[i, j] * bprob)
                #prob_beep -= (math.e ** (a*(-1)*(d-1))) ** 2 + 2 * (math.e ** (a*(-1)*(d-1)) * (1 - (math.e ** (a*(-1)*(d-1))))) * ((probgrid_m1[i, j]) ** 2)
    #Probability that both bots set off a beep
    pBoth = 0
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            pBoth += probgrid_m1[i, j] * math.e ** (a*(-1)*(outDist-1)) * probgrid_m1[x, y] * math.e ** (a*(-1)*(inDist-1))
    prob_beep_final = prob_beep - pBoth
    #Marginalize over mouse 2
    for i in range(len(probgrid_m1[0])):
        for j in range(len(probgrid_m1[0])):
            if probgrid_m1[i, j] != 0:
                newPij = 0
                outDist = abs(i - botrow) + abs(j - botcol)
                for x in range(len(probgrid_m1[0])):
                    for y in range(len(probgrid_m1[0])):
                        #print((outDist, inDist))
                        if probgrid_m1[x, y] != 0:
                            inDist = abs(x - botrow) + abs(y - botcol)
                            if beep == 1:
                                p_m1andm2_given_B = (1 - (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1)))) * probgrid_m1[i, j] * probgrid_m1[x, y] / prob_beep_final
                                newPij += p_m1andm2_given_B
                            else:
                                #Set m1 probs to prob m1 and m2 given no beep
                                p_m1andm2_given_NB = (1 - math.e ** (a*(-1)*(outDist-1))) * (1 - math.e ** (a*(-1)*(inDist-1))) * probgrid_m1[i, j] * probgrid_m1[x, y] / (1 - prob_beep_final)
                                newPij += p_m1andm2_given_NB

                    newgrid_m1[i, j] = newPij

    return newgrid_m1

#Update the probgrid when a bot moves to a cell and the bot isn't there
#Updates each P(Ai) to P(Ai|~M) = P(Ai ^ ~M) / P(~M) = P(Ai) / (1 - P(M))
def move(probgrid, moverow, movecol):
    #Probability that the mouse was in the cell we moved to
    moveprob = probgrid[moverow, movecol]
    newgrid = np.zeros((len(probgrid[0]), len(probgrid[0])))
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if probgrid[i, j] != 0:
                prevprob = probgrid[i, j]
                newgrid[i, j] = prevprob / (1 - moveprob)
    newgrid[moverow, movecol] = 0
    ##print("sum: ", np.sum(newgrid))
    return newgrid

def mouse_availability(mouserow, mousecol, ship):
    available_mouse_cells = [(mouserow, mousecol)]
    if mouserow > 0:
        if ship[mouserow-1, mousecol] == 0:
            available_mouse_cells.append((mouserow-1, mousecol))
    #check down
    if mouserow < len(ship[0])-1:
        if ship[mouserow+1, mousecol] == 0:
            available_mouse_cells.append((mouserow+1, mousecol))
    #check left
    if mousecol > 0:
        if ship[mouserow, mousecol-1] == 0:
            available_mouse_cells.append((mouserow, mousecol-1))
    #check right
    if mousecol < len(ship[0])-1:
        if ship[mouserow, mousecol+1] == 0:
            available_mouse_cells.append((mouserow, mousecol+1))
    return available_mouse_cells

def mousemove(probgrid, ship):
    newprobs = probgrid.copy()
    d = len(probgrid[0])
    #Build available_moves matrix
    available_moves = np.zeros_like(probgrid)
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            if ship[i, j] != 1:
                k = check_adj(i, j, ship, len(ship[0]))
                available_moves[i, j] = k + 1
    for i in range(len(probgrid[0])):
        for j in range(len(probgrid[0])):
            #Obtain valid neighbors of ship[i, j]
            if ship[i, j] != 1  and ship[i, j] != 2:
                neighbors = []
                if i > 0:
                    if ship[i-1, j] != 1:
                        neighbors.append((i-1, j))
                #check down
                if i < d-1:
                    if ship[i+1, j] != 1:
                        neighbors.append((i+1, j))
                #check left
                if j > 0:
                    if ship[i, j-1] != 1:
                        neighbors.append((i, j-1))
                #check right
                if j < d-1:
                    if ship[i, j+1] != 1:
                        neighbors.append((i, j+1))
                for n in neighbors:
                    #For each neighbor n of [i, j], add the probability that the mouse was in n and moved to [i, j] P(mouse in n) * P(mouse moves to (i, j))
                    #and subtract the probability that the mouse was in [i, j] and moved to n P(mouse in (i, j)) * P(mouse moves to n)
                    if available_moves[n[0], n[1]] > 0:
                        newprobs[i, j] += probgrid[n[0], n[1]] / available_moves[n[0], n[1]]
                    if available_moves[i, j] > 0:
                        newprobs[i, j] -= probgrid[i, j] / available_moves[i, j]
    return newprobs



In [100]:
#bot 3 which uses the model to dictate the bot's movements
def bot3_model(dim, a, mouse_behavior, num_mice, const_res):
    ship = const_res[0]
    botrow = const_res[1]
    botcol = const_res[2]
    mouserow = const_res[3]
    mousecol = const_res[4]
    mfound = 0
    probgrid = probbuild(ship)
    #print(ship)
    #print(probgrid)
    found = 0
    round = 1
    t = 0
    micefound = 0
    while(found == 0):
        #Loop: get action from giving model ship and probgrid, perform action, then do mouse move
        ship[mouserow, mousecol] = 0
        ship[botrow, botcol] = 0
        torchShip = torch.from_numpy(ship)
        torchProb = torch.from_numpy(probgrid)
        torchBot = torch.zeros(dim, dim)
        torchBot[botrow, botcol] = 1
        x = torch.zeros(1, dim, dim)
        x = torch.cat((torchShip, torchProb))
        xp = torch.zeros(1, 3, dim, dim)
        xp[0][0] = torchShip
        xp[0][1] = torchProb
        xp[0][2] = torchBot
        a = model(xp)[0]
        action = int(torch.argmax(a))
        #print(a)
        ship[mouserow, mousecol] = 3
        ship[botrow, botcol] = 2
        goTuple = (-1, -1)
        if action == 4:
            #sense
            dist = abs(mouserow - botrow) + abs(mousecol - botcol)
            beepprob = math.e ** (a*(-1)*(dist-1))
            roll = random.random()
            roll2 = 1
            beepprob2 = 0
            t += 1
            if roll < beepprob:
                #Beep received
                #print("Beep received")
                probgrid = response(probgrid, a, botrow, botcol, 1)
            else:
                #No beep received
                #print("No beep received")
                probgrid = response(probgrid, a, botrow, botcol, 0)
            #print(probgrid)
        if action == 0:
            #go up
            if botrow > 0:
                goTuple = (botrow - 1, botcol)
        if action == 1:
            if botrow < dim-1:
                goTuple = (botrow + 1, botcol)
        if action == 2:
            if botcol > 0:
                goTuple = (botrow, botcol - 1)
        if action == 3:
            if botcol < dim - 1:
                goTuple = (botrow, botcol + 1)
        if goTuple != (-1, -1):
            if ship[goTuple[0], goTuple[1]] == 3:
                t += 1
                found += 1
                ship[goTuple[0], goTuple[1]] = 2
                ship[botrow, botcol] = 0
                #print(ship)
                #print(probgrid)
                print("Mouse found at time ", t)
                break
            else:
                ship[goTuple[0], goTuple[1]] = 2
                ship[botrow, botcol] = 0
                t += 1
                botrow = goTuple[0]
                botcol = goTuple[1]
                probgrid = move(probgrid, botrow, botcol)
                #print(ship)
                print((botrow, botcol))
                #print(probgrid)
                #print(botrow, botcol)
                #print(probgrid[botrow, botcol+1])
                #print(probgrid[botrow, botcol-1])
        #Now: mousemove
        if mouse_behavior == 1:
            if mouserow != -1 and mousecol != -1:
                #Stochastic mouse: allow the mouse to move or stay in its cell with uniform probability
                available_mouse_cells = mouse_availability(mouserow, mousecol, ship)
                #print("available mouse cellse: ", available_mouse_cells)
                mouse_move_cell = random.choice(available_mouse_cells)
                ship[mouserow, mousecol] = 0
                ship[mouse_move_cell[0], mouse_move_cell[1]] = 3
                mouserow = mouse_move_cell[0]
                mousecol = mouse_move_cell[1]
                #Now that the mouse has moved, the bot must reevaluate the probgrid considering that the mouse could have moved
                probgrid = mousemove(probgrid, ship)
            #if num_mice == 1:
                #print("Considering possibility that mouse moved")
            #else:
                #print("Considering possibility that mice moved")
            #print(probgrid)

    return t



In [101]:
botrow = randrange(40)
botcol = randrange(40)
bot_mouse_ship = constShip.copy()
#Bot = 2
while bot_mouse_ship[botrow, botcol] != 0:
    botrow = randrange(40)
    botcol = randrange(40)
bot_mouse_ship[botrow, botcol] = 2
mouserow = randrange(40)
mousecol = randrange(40)
#Mouse = 3
while bot_mouse_ship[mouserow, mousecol] != 0:
    mouserow = randrange(40)
    mousecol = randrange(40)
bot_mouse_ship[mouserow, mousecol] = 3
const_res = [bot_mouse_ship, botrow, botcol, mouserow, mousecol]
"""pg = probbuild(bot_mouse_ship)
bot_mouse_ship[mouserow, mousecol] = 0
torchShip = torch.from_numpy(bot_mouse_ship)
torchProb = torch.from_numpy(pg)
x = torch.cat((torchShip, torchProb))
xp = torch.zeros(1, 60, 30)
xp[0] = x"""
#action = model(xp)[1]
#print(action)
#print(int(torch.argmax(action)))

#Stationary:
#res = bot3_model(40, 0.5, 0, 1, const_res)

'pg = probbuild(bot_mouse_ship)\nbot_mouse_ship[mouserow, mousecol] = 0\ntorchShip = torch.from_numpy(bot_mouse_ship)\ntorchProb = torch.from_numpy(pg)\nx = torch.cat((torchShip, torchProb))\nxp = torch.zeros(1, 60, 30)\nxp[0] = x'

In [102]:

a = torch.randn(3, 2, 3)
b = torch.randn(3, 3, 2)
print(a)
print(b)
print(torch.matmul(a, b))
#3D matmul: 1st dim has to be the same (make it 3), then matmul rules for next two dims

tensor([[[ 1.4214, -1.2435,  0.6503],
         [-2.7239,  0.6332, -0.5270]],

        [[ 1.1372, -0.6560, -0.1354],
         [-1.3901, -0.5374, -1.0896]],

        [[ 0.2892,  1.8761,  0.1956],
         [ 0.3552, -0.8425, -0.0081]]])
tensor([[[ 1.5811, -0.8392],
         [ 0.4053,  1.2892],
         [ 1.7361,  1.2886]],

        [[ 1.0440,  1.5858],
         [-0.9953,  0.0453],
         [-0.4839, -2.7152]],

        [[ 1.3600,  0.2949],
         [ 0.8329, -2.6382],
         [ 0.0903,  0.7233]]])
tensor([[[ 2.8724, -1.9580],
         [-4.9650,  2.4231]],

        [[ 1.9057,  2.1412],
         [-0.3891,  0.7296]],

        [[ 1.9735, -4.7227],
         [-0.2194,  2.3215]]])


In [103]:
x_trainS = torch.load('x_trainStationarysensing.pt')
y_trainS = torch.load('y_trainStationarysensing.pt')
x_trainM = torch.load('x_trainMovingsensing.pt')
y_trainM = torch.load('y_trainMovingsensing.pt')
x_trainS = x_trainS[0:3010]
y_trainS = y_trainS[0:3010]
x_trainM = x_trainM[0:3010]
y_trainM = y_trainM[0:3010]
torch.save(x_trainS, 'x_train_Stationary.pt')
torch.save(y_trainS, 'y_train_Stationary.pt')
torch.save(x_trainM, 'x_train_Moving.pt')
torch.save(y_trainM, 'y_train_Moving.pt')